In [1]:
pass

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1606903239535_0003,pyspark3,idle,,,✔


SparkSession available as 'spark'.


In [107]:
import pandas as pd
from pyspark.sql.functions import col, regexp_replace, rank, desc
from pyspark.sql.window import Window
import pyspark.sql.functions as F

In [3]:
date_processed_on = '2020-11-17'
path_zcode_required = 's3://zeta-dc-ml/ai-audiences/z_ai_step_codes/zcodes_required_Feb_June.csv'
path_root_dest = 's3://shubh-sandbox/ai-audiences/q4-2/'
path_root = 's3://zeta-dc-ml/ai-audiences/'


In [21]:
path_root_modeling = path_root + 'modeling/dt=' + date_processed_on + '/'
path_count_item = path_root_modeling + 'z12_counts_item'
path_user_item_interactions = path_root_modeling + 'z01_user_item_interactions'
path_scores_parquet = path_root_modeling + 'z06_scores_parquet'
path_scores_meta = path_root_modeling + 'z09_scores_meta'

# path_scores_filtered = path_root_dest + 'scores/dt=' + date_processed_on
# path_meta_of_scores_filtered = path_root_dest + 'meta/dt=' + date_processed_on


## df_zcode_required

In [5]:
df_zcode_required = spark.read.option("inferSchema", True).option("header", True).csv(path_zcode_required)

In [6]:
df_zcode_required.show(n=2)

+------+--------+
|zcodes|existing|
+------+--------+
|  z388|    true|
|  z392|    true|
+------+--------+
only showing top 2 rows

In [7]:
print(f'Number of entries/rows in the dataframe are {df_zcode_required.count()}')

Number of entries/rows in the dataframe are 86

## df_item_count

In [8]:
df_item_count = spark.read.option("inferSchema", True).parquet(path_count_item + '/*.parquet')

In [9]:
df_item_count = df_item_count.join(df_zcode_required, 'zcodes', 'inner').select('zcodes', 'count')

In [10]:
df_item_count.show(n=2)

+------+--------+
|zcodes|   count|
+------+--------+
|  z341|59496191|
|  z426|76311863|
+------+--------+
only showing top 2 rows

In [11]:
pdf = df_item_count.toPandas()

In [12]:
pdf.head(2)

  zcodes     count
0   z341  59496191
1   z426  76311863

In [ ]:
pdf.to_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_item_count_input.csv', header=True, index=False)

In [13]:
pdf.shape

(86, 2)

## Filter Scores

In [14]:
df_scores = spark.read.option("inferSchema", True).parquet(path_scores_parquet + '/*.parquet')

In [15]:
df_scores = df_scores.join(df_item_count,
                           df_scores.zcode == df_item_count.zcodes,
                           'inner').select(df_scores['*'], df_item_count['count'].alias('rank_cutoff'))

In [16]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)

In [17]:
# https://stackoverflow.com/questions/44968912/difference-in-dense-rank-and-row-number-in-spark
df_scores = df_scores.withColumn('rank', rank().over(Window.partitionBy('zcode').orderBy(desc('prediction'))))

In [18]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)

In [19]:
path_root_dest

's3://shubh-sandbox/ai-audiences/q4-2/'

In [20]:
df_scores.write.parquet(path_root_dest + 'df_scores_filtered_w_rank')

In [22]:
df_scores = spark.read.option("inferSchema", True).parquet(path_root_dest + 'df_scores_filtered_w_rank/*.parquet')

In [23]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)

In [24]:
df_scores_item_counts = df_scores.groupBy('zcode').count()

In [25]:
df_scores_item_counts.printSchema()

root
 |-- zcode: string (nullable = true)
 |-- count: long (nullable = false)

In [26]:
pdf_scores_item_counts = df_scores_item_counts.toPandas()

In [27]:
pdf_scores_item_counts.head()

  zcode      count
0    z1  222647365
1  z122  222647365
2  z536  222647365
3  z108  222647365
4  z342  222647365

In [28]:
pdf_scores_item_counts.to_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_item_count_scores.csv', header=True, index=False)

# Intersection 1

In [29]:
df_input = spark.read.option("inferSchema", True).parquet(path_user_item_interactions + '/*.parquet')

In [30]:
df_input.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcodes: string (nullable = true)
 |-- count: long (nullable = true)

In [33]:
df_input.show(n=2)

+-------------------+------+-----+
|            user_id|zcodes|count|
+-------------------+------+-----+
|1993757633875086355|  z184|  828|
|1993757633875086355|  z126|12555|
+-------------------+------+-----+
only showing top 2 rows

In [31]:
df_scores = spark.read.option("inferSchema", True).parquet(path_root_dest + 'df_scores_filtered_w_rank/*.parquet')

In [32]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)

In [34]:
df_scores.show(n=2)

+-------------------+-----+----------+-----------+----+
|            user_id|zcode|prediction|rank_cutoff|rank|
+-------------------+-----+----------+-----------+----+
| 968062829977116373|   z1| 1.7110151|   32939962|   1|
|1974054384870945501|   z1| 1.6883599|   32939962|   2|
+-------------------+-----+----------+-----------+----+
only showing top 2 rows

In [35]:
df_scores = df_scores.join(df_input,
                           ((df_scores.user_id==df_input.user_id) & (df_scores.zcode==df_input.zcodes)),
                           'inner').select(df_scores['*'], df_input['count'].alias('count_interactions'))

In [36]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)
 |-- count_interactions: long (nullable = true)

In [37]:
df_scores.write.parquet(path_root_dest + 'df_intersection_input_n_scores')

In [45]:
df_scores = spark.read.option("inferSchema", True).parquet(path_root_dest + 'df_intersection_input_n_scores/*.parquet')


In [46]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)
 |-- count_interactions: long (nullable = true)

In [47]:
df_scores_intersection_item_counts = df_scores.groupBy('zcode').count()

In [48]:
df_scores_intersection_item_counts.printSchema()

root
 |-- zcode: string (nullable = true)
 |-- count: long (nullable = false)

In [49]:
pdf_scores_intersection_item_counts = df_scores_intersection_item_counts.toPandas()

In [50]:
pdf_scores_intersection_item_counts.head()

  zcode     count
0    z1  24488395
1  z122  49144531
2  z536    493229
3  z108  18491779
4  z342  39152032

In [51]:
pdf_scores_intersection_item_counts.to_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_scores_intersection_item_counts.csv', header=True, index=False)


# Filter scores at rank cutoff

In [65]:
df_scores = spark.read.option("inferSchema", True).parquet(path_root_dest + 'df_scores_filtered_w_rank/*.parquet')

In [66]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)

In [67]:
df_scores = df_scores.withColumn('filter_flag', F.when(df_scores['rank'] < df_scores['rank_cutoff'], 1).otherwise(0))

In [68]:
df_scores = df_scores.filter(df_scores['filter_flag'] == 1)

In [69]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)
 |-- filter_flag: integer (nullable = false)

In [70]:
# df_scores = df_scores.select(['user_id', 'zcode', 'prediction'])

In [71]:
df_scores = df_scores.filter(df_scores['prediction'] > 0)

In [72]:
# df_scores = df_scores.withColumn("zcodes", regexp_replace(col("zcode"), 'z', 'zp'))

In [73]:
# df_scores = df_scores.selectExpr('user_id as cookie_sizmek', 'zcodes as zcode', 'prediction as score')

In [74]:
path_root_dest

's3://shubh-sandbox/ai-audiences/q4-2/'

In [75]:
df_scores.write.parquet(path_root_dest+'df_scores_filtered_100')

In [76]:
df_scores = spark.read.option("inferSchema", True).parquet(path_root_dest + 'df_scores_filtered_100/*.parquet')

In [77]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)
 |-- filter_flag: integer (nullable = true)

In [78]:
df_scores_filtered_100_item_counts = df_scores.groupBy('zcode').count()

In [79]:
df_scores_filtered_100_item_counts.printSchema()

root
 |-- zcode: string (nullable = true)
 |-- count: long (nullable = false)

In [80]:
pdf_scores_filtered_100_item_counts = df_scores_filtered_100_item_counts.toPandas()

In [81]:
pdf_scores_filtered_100_item_counts.head()

  zcode     count
0    z1  32939963
1  z122  61862894
2  z536    517789
3  z108  21490880
4  z342  45669968

In [82]:
pdf_scores_filtered_100_item_counts.to_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_scores_filtered_100_item_counts.csv', header=True, index=False)



# Intersection 2

In [91]:
df_input = spark.read.option("inferSchema", True).parquet(path_user_item_interactions + '/*.parquet')

In [92]:
df_input.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcodes: string (nullable = true)
 |-- count: long (nullable = true)

In [93]:
df_input.show(n=2)

+-------------------+------+-----+
|            user_id|zcodes|count|
+-------------------+------+-----+
|1993757633875086355|  z184|  828|
|1993757633875086355|  z126|12555|
+-------------------+------+-----+
only showing top 2 rows

In [94]:
df_scores = spark.read.option("inferSchema", True).parquet(path_root_dest + 'df_scores_filtered_100/*.parquet')

In [95]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)
 |-- filter_flag: integer (nullable = true)

In [96]:
df_scores.show(n=2)

+-------------------+-----+----------+-----------+----+-----------+
|            user_id|zcode|prediction|rank_cutoff|rank|filter_flag|
+-------------------+-----+----------+-----------+----+-----------+
| 968062829977116373|   z1| 1.7110151|   32939962|   1|          1|
|1974054384870945501|   z1| 1.6883599|   32939962|   2|          1|
+-------------------+-----+----------+-----------+----+-----------+
only showing top 2 rows

In [97]:
df_scores = df_scores.join(df_input,
                           ((df_scores.user_id==df_input.user_id) & (df_scores.zcode==df_input.zcodes)),
                           'inner').select(df_scores['*'], df_input['count'].alias('count_interactions'))

In [98]:
df_scores.write.parquet(path_root_dest + 'df_intersection_input_n_filtered_scores')

In [99]:
df_scores = spark.read.option("inferSchema", True).parquet(path_root_dest + 'df_intersection_input_n_filtered_scores/*.parquet')


In [100]:
df_scores.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- zcode: string (nullable = true)
 |-- prediction: float (nullable = true)
 |-- rank_cutoff: long (nullable = true)
 |-- rank: integer (nullable = true)
 |-- filter_flag: integer (nullable = true)
 |-- count_interactions: long (nullable = true)

In [101]:
df_filtered_scores_intersection_item_counts = df_scores.groupBy('zcode').count()

In [102]:
df_filtered_scores_intersection_item_counts.printSchema()

root
 |-- zcode: string (nullable = true)
 |-- count: long (nullable = false)

In [103]:
pdf_filtered_scores_intersection_item_counts = df_filtered_scores_intersection_item_counts.toPandas()

In [104]:
pdf_filtered_scores_intersection_item_counts.head()

  zcode     count
0    z1  17438243
1  z122  34442208
2  z536     39194
3  z108  10941195
4  z342  23874397

In [105]:
pdf_filtered_scores_intersection_item_counts.to_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_filtered_scores_intersection_item_counts.csv', header=True, index=False)



# collate all CSVs

In [152]:
pdf_1 = pd.read_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_item_count_input.csv')
pdf_1['count_1'] = pdf_1['count']
pdf_1['zcode'] = pdf_1['zcodes']
pdf_1 = pdf_1.set_index('zcode')
pdf_1 = pdf_1[['count', 'count_1']]
print(pdf_1.shape)
pdf_1.head(2)

(86, 2)
          count   count_1
zcode                    
z341   59496191  59496191
z426   76311863  76311863

In [153]:
pdf_2 = pd.read_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_item_count_scores.csv')

pdf_2['count_2'] = pdf_2['count']
pdf_2 = pdf_2.set_index('zcode')
print(pdf_2.shape)
pdf_2.head(2)

(86, 2)
           count    count_2
zcode                      
z1     222647365  222647365
z122   222647365  222647365

In [154]:
pdf_3 = pd.read_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_scores_intersection_item_counts.csv')

pdf_3['count_3'] = pdf_3['count']
pdf_3 = pdf_3.set_index('zcode')
print(pdf_3.shape)
pdf_3.head(2)

(86, 2)
          count   count_3
zcode                    
z1     24488395  24488395
z122   49144531  49144531

In [155]:
pdf_4 = pd.read_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_scores_filtered_100_item_counts.csv')

pdf_4['count_4'] = pdf_4['count']
pdf_4 = pdf_4.set_index('zcode')
print(pdf_4.shape)
pdf_4.head(2)

(86, 2)
          count   count_4
zcode                    
z1     32939963  32939963
z122   61862894  61862894

In [156]:
pdf_5 = pd.read_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_filtered_scores_intersection_item_counts.csv')

pdf_5['count_5'] = pdf_5['count']
pdf_5 = pdf_5.set_index('zcode')
print(pdf_5.shape)
pdf_5.head(2)

(85, 2)
          count   count_5
zcode                    
z1     17438243  17438243
z122   34442208  34442208

In [197]:
pdf_final = pd.concat([pdf_1, pdf_2, pdf_3, pdf_4, pdf_5], axis=1, sort=False, join='outer')

In [198]:
pdf_final = pdf_final[['count_1', 'count_2', 'count_3', 'count_4', 'count_5']]

In [199]:
pdf_final = pdf_final.sort_values('count_1', ascending=False)

In [200]:
pdf_final.index.name = 'zcode'

In [201]:
pdf_final = pdf_final.reset_index()


In [202]:
pdf_final.shape

(86, 6)

In [203]:
pdf_final.head()

  zcode    count_1    count_2    count_3    count_4      count_5
0  z324  242331760  222647365  160269734  222646877  160269726.0
1  z387  186471393  222647365  140253562  186471398  136112081.0
2  z380  131821245  222647365   93762607  131821255   77765189.0
3  z439  120597091  222647365   83054384  120597102   71842752.0
4  z425   89567752  222647365   74705053   89567765   55652837.0

In [204]:
pdf_final.to_csv('s3://shubh-sandbox/ai-audiences/q4-2/df_final_counts.csv', header=True, index=False)

